In [1]:
import numpy as np
import setigen as stg
from blimpy import Waterfall
import matplotlib.pyplot as plt
import random
import os
from astropy import units as u
from tqdm import tqdm
from sklearn.metrics import silhouette_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

os.environ["CUDA_VISIBLE_DEVICES"]="1"



2023-08-24 10:03:28.378008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def painting(data):
    all_data = []
    labels = []
    for c in range(num_classes):
        drift = 2*random.random()*(-1)**random.randint(0,2)
        snr = random.randint(100, 150)
        width = random.randint(20, 50)
        for s in range(num_samples_per_class):
            index = random.randint(0, data.shape[0]-1)
            window = data[index, :,:]
            
            start = random.randint(50, 180)
            
            frame = stg.Frame.from_data(df=2.7939677238464355*u.Hz,
                                        dt=18.253611008*u.s,
                                        fch1=1289*u.MHz,
                                        ascending=True,
                                        data=window)
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            all_data.append(frame.data)
            labels.append(c)
    all_data = np.array(all_data)
    labels = np.vstack(labels)
    return all_data, labels

In [3]:
import cv2
import numpy as np

def transform(data):
    final = []
    for i in range(data.shape[0]):
        img = np.repeat(data[i,:, :], 3, axis=2)
        res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        final.append(res)
    return np.array(final)


In [4]:
from tqdm import tqdm
import gc

def normalize(data):
    epsilon = 1
    min_val = data.min()
    data = data - min_val + epsilon
    new_data = np.log(data)
    min_val = data.min()
    max_val = data.max()
    final_data = (data - min_val) / (max_val - min_val)
    return final_data
    
def normalize_data(data):
    for i in range(data.shape[0]):
        data[i,:,:] = normalize(data[i,:,:, :])
    return data

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

model = ResNet50(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000
                )
truncated_model = Model(inputs = model.layers[0].input, outputs = model.layers[-1].output)
truncated_model.summary()
truncated_model.compile(optimizer='rmsprop')



2023-08-24 10:03:30.605290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 10:03:30.995300: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13888 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:61:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
def idealized(num=10_000):
    drift = drift = (-1)**(random.randint(0,2)) * 2 *random.random()
    snr = 50*random.random() +20
    width = 50*random.random() +20
    start = random.randint(50, 180)
    data = []
    labels = []
    for tag in range(4):
        label_vec = np.zeros(4)
        label_vec[tag] = 1
        for i in range(num):
            if tag == 0:
#             everything but drif
                snr = 50*random.random() +20
                width = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 1:
#             everything but snr
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                width = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 2:
#             everything but width
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                snr = 50*random.random() +20
                start = random.randint(50, 180)
            elif tag == 3:
#             everything but start
                drift = (-1)**(random.randint(0,2)) * 2 *random.random()
                snr = 50*random.random() +20
                width = 50*random.random() +20

            frame = stg.Frame(fchans=256*u.pixel,
                              tchans=16*u.pixel,
                              df=2.7939677238464355*u.Hz,
                              dt=18.253611008*u.s,
                              fch1=6095.214842353016*u.MHz)
            noise = frame.add_noise(x_mean=1, noise_type='chi2')
            frame.add_signal(stg.constant_path(
                                        f_start=frame.get_frequency(index=start),
                                       drift_rate=drift*u.Hz/u.s),
                                      stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
                                      stg.gaussian_f_profile(width=width*u.Hz),
                                      stg.constant_bp_profile(level=1))
            data.append(frame.data)
            labels.append(label_vec)
    data = np.array(data)
    labels = np.vstack(labels)
    return data, labels

In [6]:
def absolute_diff(X, labels):
    new_x = []
    new_labels = []
    one = np.arange(0,10000)
    two = np.arange(10000,20000)
    three = np.arange(20000,30000)
    four = np.arange(30000,40000)
    for i in range(X.shape[0] -1):
        if np.argmax(labels[i,:]) == np.argmax(labels[i+1,:]): 
            index = np.argmax(labels[i,:])
            if index == 0:
                sample = np.random.choice(one, size = 1000)
            elif index == 1:
                sample = np.random.choice(two, size = 1000)
            elif index == 2:
                sample = np.random.choice(three, size = 1000)
            elif index == 3:
                sample = np.random.choice(four, size = 1000)
            diff = abs(X[i,:] - X[sample,:])
            diff = np.mean(diff, axis = 0)
            new_x.append(diff)
            new_labels.append(labels[i,:])
            
    return np.array(new_x), np.vstack(new_labels)

In [7]:
import os
from tqdm import tqdm
import keras
total_scores = []
for i in tqdm(range(10)):
    idealized_signals, labels = idealized(num = 10_000)

    input_data = transform(np.expand_dims(idealized_signals, axis = -1))
    gc.collect()
    input_data = normalize_data(input_data)
    print(input_data[0,:,:].max(), input_data[0,:,:].min())
    splits = input_data.shape[0]//1_000
    X = []
    for i in range(splits):
        tensor = tf.convert_to_tensor(input_data[i*1_000:(i+1)*1_000, :,:,:], dtype=tf.float32)
        X.append(truncated_model(tensor))
    X = np.vstack(X)
    print(X.shape)
    X, labels = absolute_diff(X, labels)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

    linear = keras.Sequential(name="my_sequential")
    # linear.add(layers.Dense(1000, activation="linear", name="layer1"))
    linear.add(layers.Dense(4, activation="softmax", name="layer2"))

    linear.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    linear.fit(X_train, y_train, epochs=150, batch_size=128, validation_split = 0.1, shuffle=True)
    results = linear.evaluate(X_test, y_test)
    print(results)
    total_scores.append(results[-1])

  0%|                                                                                              | 0/10 [00:00<?, ?it/s]

1.0 0.0


2023-08-24 10:08:47.771197: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-08-24 10:08:49.747602: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-24 10:08:49.747684: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.88GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-24 10:08:49.876297: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

(40000, 1000)
Epoch 1/150


2023-08-24 10:10:21.609719: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2b940077b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-24 10:10:21.609835: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2023-08-24 10:10:21.614405: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-24 10:10:22.206395: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

[1.163940668106079, 0.7691491842269897]
1.0 0.0
(40000, 1000)
Epoch 1/150
189/189 [==============================] - 1s 2ms/step - loss: 1.3847 - accuracy: 0.2632 - val_loss: 1.3834 - val_accuracy: 0.2425
Epoch 2/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3814 - accuracy: 0.2541 - val_loss: 1.3802 - val_accuracy: 0.2425
Epoch 3/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3781 - accuracy: 0.3129 - val_loss: 1.3770 - val_accuracy: 0.2425
Epoch 4/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3749 - accuracy: 0.2811 - val_loss: 1.3736 - val_accuracy: 0.4086
Epoch 5/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3716 - accuracy: 0.3235 - val_loss: 1.3703 - val_accuracy: 0.4836
Epoch 6/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3684 - accuracy: 0.4760 - val_loss: 1.3670 - val_accuracy: 0.4832
Epoch 7/150
189/189 [==============================] - 0s 2ms/step - loss: 1.365

 90%|████████████████████████████████████████████████████████████████████████████▌        | 9/10 [52:36<05:11, 311.66s/it]

[1.0433098077774048, 0.7974846363067627]
1.0 0.0
(40000, 1000)
Epoch 1/150
189/189 [==============================] - 1s 2ms/step - loss: 1.3856 - accuracy: 0.2896 - val_loss: 1.3851 - val_accuracy: 0.2646
Epoch 2/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3840 - accuracy: 0.3090 - val_loss: 1.3837 - val_accuracy: 0.2683
Epoch 3/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3824 - accuracy: 0.2971 - val_loss: 1.3821 - val_accuracy: 0.2731
Epoch 4/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3809 - accuracy: 0.3259 - val_loss: 1.3805 - val_accuracy: 0.2948
Epoch 5/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3793 - accuracy: 0.3388 - val_loss: 1.3790 - val_accuracy: 0.2881
Epoch 6/150
189/189 [==============================] - 0s 2ms/step - loss: 1.3777 - accuracy: 0.3155 - val_loss: 1.3775 - val_accuracy: 0.3108
Epoch 7/150
189/189 [==============================] - 0s 2ms/step - loss: 1.37

100%|████████████████████████████████████████████████████████████████████████████████████| 10/10 [57:39<00:00, 345.97s/it]

[1.2038120031356812, 0.6646715402603149]


In [8]:
print(np.mean(total_scores))
print(np.std(total_scores))

0.6612546443939209
0.06619925734983738
